In [1]:
from cot import Collection
from cot.generate import FRAGMENTS
from pprint import pprint
import json

# Model Chain of Thought (CoT) Generation

## Examples of available fragments
Three groups of templates for CoT generation:
1) Instructions
2) CoT-Triggers
3) Answer-Extractions

In [2]:
pprint(FRAGMENTS["instructions"])

{'qa-01': 'Answer the following question through step-by-step reasoning.',
 'qa-02': 'Answer the following question through careful, concise step-by-step '
          'reasoning.',
 'qa-03': 'Answer the following question through careful, concise step-by-step '
          'reasoning. Avoid making up wrong statements. If the question does '
          'not make sense or cannot be answered, write "I cannot answer the '
          'question". If you do not have a good answer, write "I do not have a '
          'good answer". If you are uncertain, write "I am uncertain about '
          'this".',
 'qa-04': 'Answer the following question through careful, concise step-by-step '
          'reasoning. Avoid making up wrong statements. Generate sub-questions '
          'that are required to answer the original question, answer them '
          'until you can answer the original question. If the question does '
          'not make sense or cannot be answered, write "I cannot answer the '
          

In [3]:
print(json.dumps(FRAGMENTS["cot_triggers"], sort_keys=True, indent=2))

{
  "kojima-01": "Answer: Let's think step by step.",
  "kojima-02": "Answer: We should think about this step by step.",
  "kojima-03": "Answer: First,",
  "kojima-04": "Answer: Before we dive into the answer,",
  "kojima-05": "Answer: Proof followed by the answer.",
  "kojima-06": "Answer: Let's think step by step in a realistic way.",
  "kojima-07": "Answer: Let's think step by step using common sense and knowledge.",
  "kojima-08": "Answer: Let's think like a detective step by step.",
  "kojima-09": "Answer: Let's think about this logically.",
  "kojima-10": "Answer: Let's think step by step. First,",
  "kojima-11": "Answer: Let's think",
  "kojima-12": "Answer: Let's solve this problem by splitting it into steps.",
  "kojima-13": "Answer: The answer is after the proof.",
  "kojima-14": "Answer: Let's be realistic and think step by step.",
  "lievin-01": "Answer: Let's derive the differential diagnosis step by step.",
  "lievin-02": "Answer: Let's use step by step inductive reasonin

In [4]:
pprint(FRAGMENTS["answer_extractions"])

{'kojima-01': 'Therefore, the answer is',
 'kojima-02': 'Therefore,',
 'kojima-03': 'The answer is',
 'kojima-A-C': 'Therefore, among A through C, the answer is',
 'kojima-A-D': 'Therefore, among A through D, the answer is',
 'kojima-A-E': 'Therefore, among A through E, the answer is',
 'kojima-A-F': 'Therefore, among A through F, the answer is',
 'kojima-numerals': 'Therefore, the answer (arabic numerals) is',
 'kojima-yes-no': 'Therefore, the answer (Yes or No) is'}


## Generating Chain of Thought examples

In [5]:
# load dataset (detailed explanation in generate_samples notebook)
collection = Collection(["worldtree"], verbose=False)
worldtree_100_random = collection.select(split="train", number_samples=100, random_samples=True, seed=0)
worldtree_100_random

Loading worldtree...


| Name      |   Train | Valid   | Test   |
|-----------|---------|---------|--------|
| worldtree |     100 | -       | -      |

Not loaded: ['aqua', 'asdiv', 'commonsense_qa', 'entailment_bank', 'gsm8k', 'mawps', 'med_qa', 'medmc_qa', 'open_book_qa', 'pubmed_qa', 'qed', 'strategy_qa', 'svamp']

In [6]:
# you can also open an existing json
# collection = Collection.from_json("worldtree_100_dataset.json")

#### Example of config file

In [7]:
config={
    "idx_range": "all", # Determines which indices the generate_and_extract routine is applied to, Default: "all" (All items are used)
    "instruction_keys": ["qa-01"], # Determines which instructions are used from fragments.json, Default: None (no instructions are used)
    "cot_trigger_keys": ["kojima-01"], # Determines which cot triggers are used from fragments.json, Default: ["kojima-01"] (only the first trigger is used)
    "answer_extraction_keys": ["kojima-A-D"], # Determines which answer extraction prompts are used from fragments.json, Default: ["kojima-01"] (only the first prompt is used)
    "author" : "konstantin", # Name of the person responsible for generation, Default: ""
    "api_service": "mock_api", # Name of the API called ("openai", "huggingface_hub", or a mock for testing: "mock_api"), Default: "huggingface_hub"
    "engine": "", # Name of the engine used (for "huggingface_hub" use for example "google/flan-t5-xl"), Default: "google/flan-t5-xl"
    "temperature": 0, # Name of the person responsible for generation, Default: 0
    "max_tokens": 512, # Maximum length of output generated by the model, Default: 128
    "api_time_interval": 1.0, # Pause between two api calls in seconds, Default: 1.0
    "verbose": False, # Determines whether the progress of the generation is printed, Default: True
    "warn": True, # Determines whether a warnings that external APIs will be called are printed, Default: True
}

In [8]:
# Generating chains of thought and answer extractions (Debug mode, not calling model over API)
worldtree_100_random.generate(name="worldtree", config=config)


        You are about to  call an external API  in total 200 times, which  may produce costs .
        Number API calls for CoT generation: n_samples 100 * n_instruction_keys 1 * n_cot_trigger_keys 1
        Number API calls for answer extraction: n_samples 100 * n_instruction_keys 1 * n_cot_trigger_keys 1 * n_answer_extraction_keys 1
        Do you want to continue? y/n
         Note: You are using a mock api. When entering 'y', a test run without API calls is made. 


In [9]:
# the above was a fake call in debug mode. Now loading prepared dataset with real model answers.
worldtree_100_random = Collection.from_json("worldtree_100_generate.json")

#### Question, choices and right answer

In [10]:
pprint(worldtree_100_random["worldtree"]["train"][0]["question"])
pprint(worldtree_100_random["worldtree"]["train"][0]["choices"])
pprint(worldtree_100_random["worldtree"]["train"][0]["answer"])

('A parent and a child share several characteristics. Both individuals are '
 'tall, have curly hair, are good cooks, and have freckles. Which of these '
 'characteristics is a learned behavior?')
['being tall', 'having curly hair', 'being a good cook', 'having freckles']
['being a good cook']


#### Model generated chain of thought and extracted answer (correct)

In [11]:
pprint(worldtree_100_random["worldtree"]["train"][0]["generated_cot"][0]["cot"])
pprint(worldtree_100_random["worldtree"]["train"][0]["generated_cot"][0]['answers'][0]['answer'])

('Being a good cook is a learned behavior. The parent and the child share '
 'several characteristics. So, the answer is C.')
'C.'


### Save generated CoTs and extracted answers

In [12]:
# Saving into collection file, appending model output
worldtree_100_random.dump("worldtree_100_generate.json")

Creating json from Arrow format:   0%|          | 0/1 [00:00<?, ?ba/s]